In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
WINDOW_SIZE = 42
RANDOM_SEED = 42 # NEVER CHANGE THIS
NUM_FOLDS = 5
import numpy as np
np.random.seed(RANDOM_SEED)

In [2]:
class ProjectData(Dataset):
    def __init__(self,folder,files,window_size,sensors):
        self.sensors = sensors
        self.window_size = window_size
        self.files = files
        self.num_trials = len(self.files)
        self.num_slices = self.num_trials * [-1]
        self.trial_start_idx = self.num_trials * [-1]
        self.tables = self.num_trials * [None]
        for i,file in enumerate(self.files):
            num_lines = sum(1 for line in open(folder+file))
            num_samples = num_lines-1
            self.num_slices[i] = num_samples-window_size+1
            self.tables[i] = pd.read_csv(folder+file)
            if i == 0:
                self.trial_start_idx[i] = 0
            else:
                self.trial_start_idx[i] = self.trial_start_idx[i-1] + self.num_slices[i-1]
        self.len = sum(self.num_slices)

    def get_slice(self, trial_idx, slice):
        skiprows = lambda i: i != 0 and i < slice[0]
        nrows = slice[1]-slice[0]
        action = torch.tensor(self.tables[trial_idx].Action.values[slice[1]-1])
        contact = torch.tensor(self.tables[trial_idx].ContactMode.values[slice[1]-1])
        phase = torch.tensor(self.tables[trial_idx].Phase.values[slice[1]-1])
        sensors = torch.tensor(self.tables[trial_idx][self.sensors].values[slice[0]:slice[1],:])
        return sensors.T,action,contact,phase
    
    def find_trial_from_idx(self, idx): # a binary search algorithm
        a,b = 0,self.num_trials
        m = (a+b)//2
        while True:
            y = self.trial_start_idx[m]
            if y == idx:
                return m
            elif y > idx:
                b = m
            elif y < idx:
                a = m
            if (b-a) <= 1:
                return a
            m = (a+b)//2

    def __len__(self):
        return self.len
    
    def __getitem__(self,idx):
        if idx >= self.len:
            raise IndexError()
        trial = self.find_trial_from_idx(idx)
        slice = (idx - self.trial_start_idx[trial], idx - self.trial_start_idx[trial] + self.window_size)
        return self.get_slice(trial, slice)

SENSORS = ["foot_Accel_X",
           "foot_Accel_Y", 
           "foot_Accel_Z",
           "foot_Gyro_X",
           "foot_Gyro_Y",
           "foot_Gyro_Z",
           "shank_Accel_X",
           "shank_Accel_Y",
           "shank_Accel_Z",
           "shank_Gyro_X",
           "shank_Gyro_Y",
           "shank_Gyro_Z",
           "thigh_Accel_X",
           "thigh_Accel_Y",
           "thigh_Accel_Z",
           "thigh_Gyro_X",
           "thigh_Gyro_Y",
           "thigh_Gyro_Z",
           "trunk_Accel_X",
           "trunk_Accel_Y",
           "trunk_Accel_Z",
           "trunk_Gyro_X",
           "trunk_Gyro_Y",
           "trunk_Gyro_Z",
           "gastrocmed",
           "vastusmedialis",
           "vastuslateralis",
           "tibialisanterior",
           "rectusfemoris",
           "bicepsfemoris",
]

IMU = [ "foot_Accel_X",
        "foot_Accel_Y", 
        "foot_Accel_Z",
        "foot_Gyro_X",
        "foot_Gyro_Y",
        "foot_Gyro_Z",
        "shank_Accel_X",
        "shank_Accel_Y",
        "shank_Accel_Z",
        "shank_Gyro_X",
        "shank_Gyro_Y",
        "shank_Gyro_Z",
        "thigh_Accel_X",
        "thigh_Accel_Y",
        "thigh_Accel_Z",
        "thigh_Gyro_X",
        "thigh_Gyro_Y",
        "thigh_Gyro_Z",
        "trunk_Accel_X",
        "trunk_Accel_Y",
        "trunk_Accel_Z",
        "trunk_Gyro_X",
        "trunk_Gyro_Y",
        "trunk_Gyro_Z"
]

EMG = [
        "gastrocmed",
        "vastusmedialis",
        "vastuslateralis",
        "tibialisanterior",
        "rectusfemoris",
        "bicepsfemoris",
]

def make_folds(K,folder,files,window_size,sensors):
    _files = np.array(files)
    N = len(files)
    M = N//K
    indices = np.random.choice(N,N,replace=False)
    folds = K*[None]
    for i in range(K-1):
        folds[i] = _files[indices[i*M:(i+1)*M]]
    folds[K-1] = _files[indices[(K-1)*M:-1]]
    return [ProjectData(folder,folds[i],window_size,sensors) for i in range(K)]


training_files = os.listdir("train")

In [3]:
class ResNeXtBlock(nn.Module):
    def __init__(self, num_sensors, window_size, kernels, dilation):
        super().__init__()
        self.conv1 = nn.Conv1d(
                num_sensors,num_sensors,
                kernels[0],
                padding='same',
                dilation=dilation[0],
                groups=num_sensors)
        self.bn1 = nn.LazyBatchNorm1d()
        self.conv2 = nn.Conv1d(
                num_sensors,num_sensors,
                kernels[1],
                padding='same',
                dilation=dilation[1],
                groups=num_sensors)
        self.bn2 = nn.LazyBatchNorm1d()
        self.conv3 = nn.Conv1d(
                num_sensors,num_sensors,
                kernels[2],
                padding='same',
                dilation=dilation[2])
        self.bn3 = nn.LazyBatchNorm1d()

    def forward(self, X):
        Y = nn.functional.relu(self.bn1(self.conv1(X)))
        Y = nn.functional.relu(self.bn2(self.conv2(X)))
        Y = self.bn3(self.conv3(Y))
        return nn.functional.relu(Y+X)
    
class ResNeXt(nn.Module):
    def __init__(self, num_sensors, window_size, kernels, dilation, num_blocks):
        super().__init__()
        self.loss_weights=torch.tensor([1,1,1])
        blocks = [ResNeXtBlock(num_sensors,window_size,kernels,dilation) for i in range(num_blocks)] 
        self.blocks = nn.Sequential(*blocks)
        self.bn = nn.LazyBatchNorm1d()
        self.gap = nn.AdaptiveAvgPool1d(5)
        self.fc1 = nn.LazyLinear(5)
        self.fc2 = nn.LazyLinear(2)
        self.fc3 = nn.LazyLinear(2)
    
    def forward(self, sensors):
        # return self.blocks[0](sensors)
        Y = torch.flatten(self.gap(self.bn(self.blocks(sensors))),start_dim=1,end_dim=-1)
        action_logits = self.fc1(Y)
        contact_logits = self.fc2(Y)
        phase_xy = self.fc3(Y)
        phase = torch.atan2(phase_xy[:,0],phase_xy[:,1])
        return action_logits, contact_logits, phase + torch.pi

    def loss(self, action_logits, contact_logits, phase_pred, action, contact, phase):
        loss = self.loss_weights[0]*nn.functional.cross_entropy(action_logits, action)
        loss += self.loss_weights[1]*nn.functional.cross_entropy(contact_logits, contact)
        # convert phases to cartesian_coords
        xy = torch.vstack((torch.cos(phase),torch.sin(phase)))
        xy_pred = torch.vstack((torch.cos(phase_pred),torch.sin(phase_pred)))
        loss += self.loss_weights[2]*nn.functional.mse_loss(xy_pred,xy)
        return loss

    def to(self,device):
        super().to(device)
        self.loss_weights = self.loss_weights.to(device)
        return self


In [4]:
# testing model constructor
model = ResNeXt(
    len(SENSORS),
    WINDOW_SIZE,
    [3,3,3],
    [1,2,3],
    10
)
model = model.double()

/home/jgs/miniconda3/envs/directml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [5]:
# trains model on dataloader.dataset
def train_loop(dataloader, model, device, optimizer):
    size = len(dataloader.dataset)
    epoch_loss = 0
    for batch, (sensors,action,contact,phase) in enumerate(dataloader):
        sensors = sensors.to(device)
        action = action.to(device)
        contact = contact.to(device)
        phase = phase.to(device)

        # Compute prediction and loss
        action_logits, contact_logits, phase_pred = model(sensors)
        loss = model.loss(action_logits, contact_logits, phase_pred, action, contact, phase)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(sensors)
            # print(f"average loss: {loss/len(batch):>7f}  [{current:>5d}/{size:>5d}]")
    return epoch_loss

# tests model on dataloader.dataset
def test_loop(dataloader, model, device):
    epoch_loss = 0
    with torch.no_grad():
        for batch, (sensors,action,contact,phase) in enumerate(dataloader):
            sensors = sensors.to(device)
            action = action.to(device)
            contact = contact.to(device)
            phase = phase.to(device)
            # Compute prediction and loss
            action_logits, contact_logits, phase_pred = model(sensors)
            epoch_loss += model.loss(action_logits, contact_logits, phase_pred, action, contact, phase).item()
    return epoch_loss

In [6]:
# Runs k-fold cross-validation training for two essential hyper-parameters:
# 1) Window size (how many past measurements are used for inference)
# 2) Sensors (which signals are used for inference)
# Note that these hyperparameters change both the dataset and the network width
def kfold_training(K,window_size,sensors,epochs,dataframe):
    sensor_sets = {"EMG" : EMG, "IMU": IMU, "EMG+IMU": SENSORS}
    folds = make_folds(K,"train/",training_files,window_size,sensor_sets[sensors])
    K = len(folds)
    loss = K*[None]
    for i in range(K):
        model = ResNeXt(
            len(sensor_sets[sensors]),
            window_size,
            [3,3,3],
            [1,2,3],
            10
        )
        model = model.double()
        model = model.to("cuda")
        optimizer = torch.optim.SGD(model.parameters(),1e-3,momentum=0.9,nesterov=True)
        for t in range(epochs):
            train_loss = 0
            for j in range(K):
                if j == i:
                    continue
                dataloader = DataLoader(folds[j],batch_size=4096,shuffle=True,num_workers=8)
                train_loss += train_loop(dataloader,model,"cuda",optimizer)
            dataloader = DataLoader(folds[i],batch_size=4096,num_workers=8)
            validation_loss = test_loop(dataloader, model, "cuda")
            df = pd.DataFrame({
                "Window Size": window_size,
                "Sensors": sensors,
                "Fold": i,
                "Epoch": t,
                "Train Loss": [train_loss],
                "Validation Loss": [validation_loss]
            })
            dataframe = pd.concat((dataframe,df))
            print(dataframe)
    return dataframe


In [7]:
# Run 5-fold cross-validation over 9 sets of hyper parameters
# window_sizes = [10,25,75]
# sensor_sets = ["EMG","IMU","EMG+IMU"]
# epochs = 5
# df = pd.DataFrame(columns = ["Window Size", "Sensors", "Fold", "Epoch", "Train Loss", "Validation Loss"])
# for window_size in window_sizes:
#     for sensors in sensor_sets:
#         df = kfold_training(5,window_size,sensors,epochs,df)

/home/jgs/miniconda3/envs/directml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


  Window Size Sensors Fold Epoch   Train Loss  Validation Loss
0          10     EMG    0     0  1000.478677       233.516736
  Window Size Sensors Fold Epoch   Train Loss  Validation Loss
0          10     EMG    0     0  1000.478677       233.516736
0          10     EMG    0     1   902.974508       226.947823
  Window Size Sensors Fold Epoch   Train Loss  Validation Loss
0          10     EMG    0     0  1000.478677       233.516736
0          10     EMG    0     1   902.974508       226.947823
0          10     EMG    0     2   882.501823       223.639982


KeyboardInterrupt: 

In [ ]:
# put the results into a dataframe

# import seaborn as sns
# import seaborn.objects as so
# from matplotlib import pyplot as plt
# ax = sns.swarmplot(dataframe,x="Sensors",y="Loss",hue="window size",palette="tab10")
# ax.set_title("Five-Fold Cross-Validation Loss of Models")
# plt.savefig("resnext10 crossval study.png")

In [ ]:
# Best trade-off is IMU, so we will train the model using IMU data and 70 data samples per sensor
# Run 7-fold cross-validation for 7 epochs
# takes about 2.5 hours on my desktop computer with a cuda device
df = pd.DataFrame(columns = ["Window Size", "Sensors", "Fold", "Epoch", "Train Loss", "Validation Loss"])
df = kfold_training(7,70,"IMU",7,df)
# Overfitting seems to occur after 5 epochs

/home/jgs/miniconda3/envs/directml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


  Window Size Sensors Fold Epoch  Train Loss  Validation Loss
0          70     IMU    0     0  658.158829        74.142022
  Window Size Sensors Fold Epoch  Train Loss  Validation Loss
0          70     IMU    0     0  658.158829        74.142022
0          70     IMU    0     1  302.313886        56.625108
  Window Size Sensors Fold Epoch  Train Loss  Validation Loss
0          70     IMU    0     0  658.158829        74.142022
0          70     IMU    0     1  302.313886        56.625108
0          70     IMU    0     2  204.521188        53.057677
  Window Size Sensors Fold Epoch  Train Loss  Validation Loss
0          70     IMU    0     0  658.158829        74.142022
0          70     IMU    0     1  302.313886        56.625108
0          70     IMU    0     2  204.521188        53.057677
0          70     IMU    0     3  164.870926        52.261207
  Window Size Sensors Fold Epoch  Train Loss  Validation Loss
0          70     IMU    0     0  658.158829        74.142022
0       

/home/jgs/miniconda3/envs/directml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


  Window Size Sensors Fold Epoch  Train Loss  Validation Loss
0          70     IMU    0     0  658.158829        74.142022
0          70     IMU    0     1  302.313886        56.625108
0          70     IMU    0     2  204.521188        53.057677
0          70     IMU    0     3  164.870926        52.261207
0          70     IMU    0     4  143.671930        52.111264
0          70     IMU    0     5  130.556079        52.527088
0          70     IMU    0     6  121.121041        53.322133
0          70     IMU    1     0  680.387996        81.789701
  Window Size Sensors Fold Epoch  Train Loss  Validation Loss
0          70     IMU    0     0  658.158829        74.142022
0          70     IMU    0     1  302.313886        56.625108
0          70     IMU    0     2  204.521188        53.057677
0          70     IMU    0     3  164.870926        52.261207
0          70     IMU    0     4  143.671930        52.111264
0          70     IMU    0     5  130.556079        52.527088
0       

/home/jgs/miniconda3/envs/directml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


  Window Size Sensors Fold Epoch  Train Loss  Validation Loss
0          70     IMU    0     0  658.158829        74.142022
0          70     IMU    0     1  302.313886        56.625108
0          70     IMU    0     2  204.521188        53.057677
0          70     IMU    0     3  164.870926        52.261207
0          70     IMU    0     4  143.671930        52.111264
0          70     IMU    0     5  130.556079        52.527088
0          70     IMU    0     6  121.121041        53.322133
0          70     IMU    1     0  680.387996        81.789701
0          70     IMU    1     1  336.268502        58.041528
0          70     IMU    1     2  217.721403        50.870223
0          70     IMU    1     3  171.567056        49.115019
0          70     IMU    1     4  147.136729        48.577843
0          70     IMU    1     5  131.361790        48.870502
0          70     IMU    1     6  120.952335        49.024069
0          70     IMU    2     0  657.505088        89.189420
  Window